In [1]:
from __future__ import print_function, division

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(context="poster", style="ticks")
import ipywidgets
import yt
from matplotlib.colors import LogNorm

import glob
import os
import warnings
import h5py

import numpy as np
import pandas as pd

from units import M_solar, m_proton, pc, yr, Myr, km, s, gamma

from injection_helpers import get_SNe

from visualize_helpers import \
    get_snapshot_filenames, \
    snapshot_filename_to_number, \
    total_mass_of_snapshot, \
    total_radial_momentum_of_snapshot, \
    map_to_all_snapshots, \
    get_snapshot_times, \
    total_kinetic_energy_of_snapshot, \
    total_internal_energy_of_snapshot, \
    load_snapshots, \
    plot_projected_density, \
    plot_sliced_field, \
    plot_phase_diagram, \
    field_type, \
    plot_profile, \
    yt_plot_saver, \
    mpl_plot_saver, \
    load_ds_from_ts

import MHD
    
from sql_helpers import add_simulation, \
    open_as_DataFrame

%load_ext autoreload
%autoreload 2

/pfs/home/egentry/anaconda3/envs/sph/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


`grackle_helpers.wrapped_initializer` is setting up grackle assuming code units:
    mass   : 1.0 M_solar
    length : 1.0 pc
    time   : 1.0 Myr

grackle cooling file:  b'/pfs/home/egentry/local/grackle/input/CloudyData_UVB=HM2012.h5'
`grackle_helpers.wrapped_initializer` is setting up grackle assuming code units:
    mass   : 8.41e-58 M_solar
    length : 3.24e-19 pc
    time   : 3.17e-14 Myr

grackle cooling file:  b'/pfs/home/egentry/local/grackle/input/CloudyData_UVB=HM2012.h5'


In [2]:
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['figure.facecolor'] = "white"

In [87]:
def weighted_percentile(x, weights, percentiles):
    weights = weights.copy()
    weights /= weights.sum()

    sorted_indices = np.argsort(x)
    x = x[sorted_indices]
    weights = weights[sorted_indices]

    weights = weights.cumsum() * 100

    percentile_indices = [ np.argmin(np.abs(weights - percentile))
        for percentile in percentiles
    ]

    percentile_indices = np.array(percentile_indices)

    results = x[percentile_indices]
    return results

def get_interface_mask(temps, 
                       min_temp=3e4, max_temp=3e5,):
    mask = temps < max_temp
    mask &= temps > min_temp
    return mask

def calc_iqr(values, weights, 
            ):
    
    low, mid, high = weighted_percentile(values,
                                         weights,
                                         [25, 50, 75])
    iqr = (high - low)

    return mid, iqr

In [99]:
run_names = [
    "cluster_cooling_100",
    "cluster_cooling_200",
    "cluster_cooling_300",
    "cluster_cooling_400",
    "cluster_cooling_600",
]

results_by_run_name = {}
for run_name in run_names:
    
    run_dir = os.path.join(os.path.pardir, "runs", run_name)

    outputs_dir = os.path.join(run_dir, "outputs")
    
    ts = load_snapshots(outputs_dir)
    snapshot_filenames = get_snapshot_filenames(outputs_dir)

    snapshot_number_to_index_map = {snapshot_filename_to_number(filename) : i
                                    for i,filename in enumerate(snapshot_filenames) }

    ds = load_ds_from_ts(ts, snapshot_number_to_index_map[8])
    
    center = ds.domain_center.value[0]
    region = ds.sphere([center,center,center], (140, "pc"))
    
    radii = region["all", "particle_position_spherical_radius"].to("pc").value
    temps = region["all", "temperature"].value
    densities = region["all", "density"].value / m_proton
    masses = region["all", "mass"].to("Msun").value
    smoothing_lengths = region["all", "smoothing_length"].to("pc").value
    
    interface_mask = get_interface_mask(temps)
    
    radii = radii[interface_mask]
    temps = temps[interface_mask]
    densities = densities[interface_mask]
    masses = masses[interface_mask]
    smoothing_lengths = smoothing_lengths[interface_mask]
    
    mid, iqr = calc_iqr(radii, masses)
    
    smoothing_lengths_med = weighted_percentile(smoothing_lengths,
                                            masses,
                                            [50])[0]
    
    results = dict(
        mid=mid, iqr=iqr, m_int=masses.sum(),
        h_med=smoothing_lengths_med,
        N_particles = masses.size,
    )

    print(run_name, "r_med = {mid:.1f}, iqr = {iqr:.1f}, m_int = {m_int:.1f}, h_med = {h_med:.1f}, N_p={N_particles}".format(
        **results
    ))

    results_by_run_name[run_name] = results


yt : [INFO     ] 2018-10-01 17:44:12,025 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2018-10-01 17:44:12,070 Parameters: current_time              = 1.0148989313228363
yt : [INFO     ] 2018-10-01 17:44:12,072 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2018-10-01 17:44:12,074 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2018-10-01 17:44:12,076 Parameters: domain_right_edge         = [600. 600. 600.]
yt : [INFO     ] 2018-10-01 17:44:12,077 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2018-10-01 17:44:12,283 Allocating for 3.375e+06 particles
Initializing refined index: 100%|██████████| 13/13 [00:00<00:00, 32.29it/s]
yt : [INFO     ] 2018-10-01 17:44:21,515 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2018-10-01 17:44:21,539 Parameters: current_time              = 1.0148989313228363
yt : [INFO     ] 2018-10-01 17:44:21,541 Parameters: domain_dimensions         = [1 1 1]
yt : [I

cluster_cooling_100 r_med = 37.0, iqr = 8.5, m_int = 33.6, h_med = 15.4, N_p=16


yt : [INFO     ] 2018-10-01 17:44:22,378 Allocating for 2.700e+07 particles
Initializing refined index: 100%|██████████| 103/103 [00:05<00:00, 17.06it/s]


cluster_cooling_200 r_med = 39.1, iqr = 8.3, m_int = 41.8, h_med = 9.8, N_p=159


yt : [INFO     ] 2018-10-01 17:45:18,535 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2018-10-01 17:45:18,572 Parameters: current_time              = 1.0148989313228363
yt : [INFO     ] 2018-10-01 17:45:18,573 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2018-10-01 17:45:18,575 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2018-10-01 17:45:18,576 Parameters: domain_right_edge         = [400. 400. 400.]
yt : [INFO     ] 2018-10-01 17:45:18,578 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2018-10-01 17:45:20,088 Allocating for 2.700e+07 particles
Loading particle index: 100%|██████████| 103/103 [00:00<00:00, 115.76it/s]


cluster_cooling_300 r_med = 43.3, iqr = 6.4, m_int = 37.8, h_med = 6.6, N_p=485


yt : [INFO     ] 2018-10-01 17:46:23,130 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2018-10-01 17:46:23,151 Parameters: current_time              = 1.0148989313228363
yt : [INFO     ] 2018-10-01 17:46:23,151 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2018-10-01 17:46:23,152 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2018-10-01 17:46:23,153 Parameters: domain_right_edge         = [600. 600. 600.]
yt : [INFO     ] 2018-10-01 17:46:23,154 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2018-10-01 17:46:30,120 Allocating for 2.160e+08 particles
Loading particle index: 100%|██████████| 824/824 [00:06<00:00, 123.61it/s]
yt : [INFO     ] 2018-10-01 17:54:08,569 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2018-10-01 17:54:08,599 Parameters: current_time              = 1.0148989313228363
yt : [INFO     ] 2018-10-01 17:54:08,600 Parameters: domain_dimensions         = [1 1 1]
yt : [IN

cluster_cooling_400 r_med = 45.4, iqr = 5.1, m_int = 42.8, h_med = 4.9, N_p=1303


yt : [INFO     ] 2018-10-01 17:54:11,607 Allocating for 9.113e+07 particles
Initializing refined index: 100%|██████████| 348/348 [00:25<00:00, 13.59it/s]


cluster_cooling_600 r_med = 47.4, iqr = 4.3, m_int = 43.2, h_med = 3.3, N_p=4436


In [100]:
results_by_run_name

{'cluster_cooling_100': {'N_particles': 16,
  'h_med': 15.360845565881245,
  'iqr': 8.47609456540767,
  'm_int': 33.64785394103906,
  'mid': 37.042409950759655},
 'cluster_cooling_200': {'N_particles': 159,
  'h_med': 9.78702976067506,
  'iqr': 8.252948486443387,
  'm_int': 41.796943567384446,
  'mid': 39.09239038545648},
 'cluster_cooling_300': {'N_particles': 485,
  'h_med': 6.551852469997345,
  'iqr': 6.39524133283652,
  'm_int': 37.78781537763416,
  'mid': 43.26661075706239},
 'cluster_cooling_400': {'N_particles': 1303,
  'h_med': 4.938010337900968,
  'iqr': 5.098563648461479,
  'm_int': 42.815579770677616,
  'mid': 45.41759932242604},
 'cluster_cooling_600': {'N_particles': 4436,
  'h_med': 3.282545805514495,
  'iqr': 4.263249701020001,
  'm_int': 43.18915683596294,
  'mid': 47.381801934790495}}

In [102]:
run_name_to_title = {
"cluster_cooling_600": r"\texttt{3D\_07\_HD}",
"cluster_cooling_400": r"\texttt{3D\_10\_HD}",
"cluster_cooling_300": r"\texttt{3D\_13\_HD}",
"cluster_cooling_200": r"\texttt{3D\_20\_HD}",
"cluster_cooling_100": r"\texttt{3D\_40\_HD}",
}

In [117]:
for run_name in run_names:
    results = results_by_run_name[run_name]
    title = run_name_to_title[run_name]
    
    iqr_h = results["iqr"] / results["h_med"]
    
    print(("{title} & {mid:.1f} & {iqr:.1f} & {iqr_h:.1f}"
          r" & {h_med:.1f} & {m_int:.1f} & {N_particles}"
          r" \\").format(
        title=title, iqr_h=iqr_h,
        **results
    ))

\texttt{3D\_40\_HD} & 37.0 & 8.5 & 0.6 & 15.4 & 33.6 & 16 \\
\texttt{3D\_20\_HD} & 39.1 & 8.3 & 0.8 & 9.8 & 41.8 & 159 \\
\texttt{3D\_13\_HD} & 43.3 & 6.4 & 1.0 & 6.6 & 37.8 & 485 \\
\texttt{3D\_10\_HD} & 45.4 & 5.1 & 1.0 & 4.9 & 42.8 & 1303 \\
\texttt{3D\_07\_HD} & 47.4 & 4.3 & 1.3 & 3.3 & 43.2 & 4436 \\


In [ ]:
    results = dict(
        mid=mid, iqr=iqr, m_int=masses.sum(),
        h_med=smoothing_lengths_med,
        N_particles = masses.size,
    )

In [63]:
results_by_run_name

{'cluster_cooling_100': {'h_avg': 15.360845565881245,
  'iqr': 8.47609456540767,
  'm_int': 1.0,
  'mid': 37.042409950759655}}

In [53]:

weighted_percentile(smoothing_lengths,
                                        masses,
                                        [50])

array([15.36084557])

In [54]:
smoothing_lengths.mean()

15.235466010229786

In [48]:
region["all", "smoothing_length"].to("pc").value

array([7.88040787, 7.88003977, 7.87988608, ..., 7.88028158, 7.87998295,
       7.8803337 ])

In [45]:
region["all", "particle_radius"].to("pc")

YTArray([139.02407766, 135.82233705, 137.05473192, ..., 144.27749476,
         147.18732999, 145.0506045 ]) pc

In [46]:
region["all", "radius"].to("pc")

YTArray([139.02407766, 135.82233705, 137.05473192, ..., 144.27749476,
         147.18732999, 145.0506045 ]) pc

In [41]:
masses.size

16

In [42]:
interface_mask.sum()

16

In [30]:
interface_mass

1.0

In [27]:
mid

37.042409950759655

In [28]:
iqr

8.47609456540767